In [ ]:
import pandas as pd
import numpy as np
import json
import textdistance as td


In [ ]:
def printUniqueTokens(series):
    unique_series = series.unique()
    token_count = {}
    for a in unique_series:
        tokens = a.split(' ')
        for t in tokens:
            if t not in token_count:
                token_count[t] = 1
            else:
                token_count[t] += 1

    for key, value in sorted(token_count.items(), key=lambda item: item[1]):
        print("%s: %s" % (key, value))

In [6]:
golddupes = pd.read_csv('../data/work/restaurants_DPL.tsv', delimiter='\t')
dupedict = {}
for i, dupeRow in golddupes.iterrows():
    if dupeRow[0] not in dupedict:
        dupedict[dupeRow[0]] = set()
        dupedict[dupeRow[0]].add(dupeRow[0])
    #dupedict[dupeRow[0]].add(dupeRow[0])
    dupedict[dupeRow[0]].add(dupeRow[1])
    
dupesets = list(dupedict.values())
dupeids = [y for x in dupesets for y in x]

NameError: name 'pd' is not defined

In [4]:
def compareToGold(dfparam):
    recdupes = list(dfparam[dfparam.id.map(len) > 1].id)
    true_positive = []
    false_negative = []
    false_positive = []
    for dupeset in dupesets:
        if dupeset in recdupes:
            true_positive.append(dupeset)
        else:
            false_negative.append(dupeset)
    for recset in recdupes:
        if recset not in dupesets:
            false_positive.append(recset)
    print("True positives: " + str(len(true_positive)))
    print("False negatives: " + str(len(false_negative)))
    print("False positives: " + str(len(false_positive)))
    precision = len(true_positive) / (len(true_positive) + len(false_positive))
    recall = len(true_positive) / (len(true_positive) + len(false_negative))
    fscore = (2*precision*recall) / (precision+recall)
    print("Precision: " + str(precision))
    print("Recall: " + str(recall))
    print("Fscore: " + str(fscore))
    return true_positive, false_negative, false_positive

In [ ]:
# Sauce: https://www.geeksforgeeks.org/python-merge-list-with-common-elements-in-a-list-of-lists/
#Importing  
from collections import defaultdict 
  
#merge function to  merge all sublist having common elements. 
def merge_common(lists): 
    neigh = defaultdict(set) 
    visited = set() 
    for each in lists: 
        for item in each: 
            neigh[item].update(each) 
    def comp(node, neigh = neigh, visited = visited, vis = visited.add): 
        nodes = set([node]) 
        next_node = nodes.pop 
        while nodes: 
            node = next_node() 
            vis(node) 
            nodes |= neigh[node] - visited 
            yield node 
    for node in neigh: 
        if node not in visited: 
            yield sorted(comp(node)) 

In [ ]:
def cleanGood(df):
    data = df.copy()
    data.type = data.type.fillna("")
    cityDict = {
        "w. hollywood": "west hollywood",
        "new york city": "new york",
        "west la": "los angeles",
        "la": "los angeles"
    }

    addressDict = {
        r"(ave|av)": "ave",
        r"(blvd|blv)": "blvd",
        r"(sts)": "st",
        r"s\.": "s",
        r" ?between.*$": ""
    }

    BRACKETS_REGEX = r" ?\(.*\)"
    NON_ALPHA_OR_SPACE_REGEX = r"[^a-zA-Z0-9 ]"
    NON_ALPHA_REGEX = r"[^a-zA-Z0-9]"
    def preProcess(origData):
        df = origData.copy()
        #df = df.drop(labels=['id'], axis=1)
        #df.set_index("id", inplace = True)

        for k, v in addressDict.items():
            df.address = df.address.str.replace(k, v, case=False, regex=True)
        for k, v in cityDict.items():
            df.city = df.city.str.replace(k, v, case=False)
        df.type = df.type.fillna("")
        df.type = df.type.str.replace(BRACKETS_REGEX, '', case=False)\
            .str.replace(r"^.*[0-9] ?", "", case=False)
        df.phone = df.phone.str.replace(NON_ALPHA_REGEX, '', case=False)
        df.name = df.name.str.replace(BRACKETS_REGEX, '', case=False)

        df["cname"] = df.name.copy()
        df["caddress"] = df.address.copy()
        df.cname = df.cname.str.replace(NON_ALPHA_OR_SPACE_REGEX, '', case=False)\
            .str.replace('  the$', '', case=False)
        df.caddress = df.caddress.str.replace(NON_ALPHA_OR_SPACE_REGEX, '', case=False)

        #df = df.drop_duplicates(["city", "phone", "type", "name", "address"])
        return df
    preProcessedData = preProcess(data)    
    groupedData = preProcessedData.groupby(["city", "cname", "caddress"]).agg(set).reset_index()
    
    from itertools import combinations

    def bias(p1, p2):
        if p1 in p2 or p2 in p1:
            return 1
        else:
            return 0

    def calcDistances(nameset):
        alldists = []

        combis = set(map(frozenset, combinations(set(nameset), 2)))
        #for c in combis:
        dists = []
        dists = [(p1, p2, p1 == p2 and -1 or td.jaccard(p1, p2) + bias(p1, p2)) for p1, p2 in combis]
        alldists += dists

        #for someset in nameseries:
        #    if len(someset) <= 1:
        #        continue
        #    dists = []
        #    dists = [(p1, p2, p1 == p2 and -1 or td.jaccard(p1, p2) + bias(p1, p2)) for p1 in someset for p2 in someset]
        #    alldists += dists
        alldists.sort(key=lambda x: x[2], reverse=True)
        return alldists

    alldists = calcDistances(preProcessedData.cname.unique())
    
    def transformToEqualityRings(l):
        sets = map(lambda x: set(x[0:2]), l)
        return list(merge_common(sets)) 

    filtered = list(filter(lambda x: x[2] >= 0.70, alldists))
    filtered
    eqRing = transformToEqualityRings(filtered)
    #uniquemappings = dict(map(lambda x: set(x[0:2]), filtered))
    
    def findClosestMatch(x):
        for p1, p2, v in filtered:
            if x == p1:
                return p1
            elif x == p2:
                return p1
        return x

    def findClosestEqMatch(x):
        for ring in eqRing:
            for e in ring:
                if e == x:
                    return ring[0]
        return x

    def mapTextDistanceToUnique(df):
        df["tdkey"] = df.cname.copy()
        #df.tdkey = df.tdkey.apply(findClosestMatch)
        df.tdkey = df.tdkey.apply(findClosestEqMatch)
        return df

    tdData = preProcessedData.copy()
    tdData = mapTextDistanceToUnique(tdData)
    tdData
    
    dedupeData2 = tdData.groupby(["phone", "tdkey"]).agg(set).reset_index()
    print("# SIMPLE GROUP")
    compareToGold(groupedData)
    print("# GROUP BY PHONE")
    a, b, c = compareToGold(dedupeData2)
    missing = [y for x in b for y in x]
    print(tdData[tdData.id.isin(missing)])
    return dedupeData2